This notebook was prepared by [SentinelWarren](https://github.com/sentinelwarren). Source and license info is on [GitHub](https://github.com/sentinelwarren/patterns-in-various-langs).

# Singleton Pattern

## Description

Is a way to provide one and only one object of a particular type. The Singleton class can have only one instance, and provides global access to that one instance.

Due to its instantiation restrictions of a class to one object, this pattern is useful when you need one object to coordinate actions for the system.

The basic idea is that only one instance of a particular class, doing a job, is created for the needs of the program. To ensure that this works, we need mechanisms that prevent the instantiation of the class more than once and also prevent cloning.

## Singleton in the wild

Singleton pattern can be used to implement projects that do not affect the execution of code, hence its acceptable to use global state. The kinds of projects are;

* Logging
* Caching
* Load balancing
* Route mapping

In all these cases,information flows in one direction, and the singleton instance itself is immutable (it does not change).
No part of the program attempts to make a change in the singleton, and as such there is no danger of one part of a project interfering with another part of the project because of the shared state.

### Example projects

* Plone CMS

## Pros & Cons

### Pros

* Access to global state, making a Singleton instance to be immutable.

### Cons | Drawbacks

* Possibility of coupling your code to itself in very unfortunate ways, causing unexpected results in completely unrelated piece of code.

## Use cases

* Maintaining a global state of a program

* Controlling concurrent access to a shared resource. For example, the class managing the connection to a database.

* A service or resource that is transversal in the sense that it can be accessed from different parts of the application or by different users and do its work. For example, the class at the core of the logging system or utility.

## Implementation

* Using metaclass and its type, having predefined the `Singleton` metaclass.
* As required, the metaclass's `__call__()` method holds the code that ensures that only one instance of the class can be created.

### Test cases

* `len(SingletonClass().instances) == 0`
* `SingletonClass() == SingletonClass()`
* `SingletonClass().attributes == SingletonClass().attributes`
* `len(SingletonClass().instances) == 1`

In [2]:
%%writefile singleton.py
from typing import Dict


class Singleton(type):
    _instances: Dict[str, object] = dict()

    def __call__(cls, *args: object, **kwargs: object) -> object:
        if str(cls) not in cls._instances:
            cls._instances[str(cls)] = super(Singleton,
            cls).__call__(*args, **kwargs)

        return cls._instances[str(cls)]


class FooTone(metaclass=Singleton):
    def __init__(self, arg: str) -> None:
        self.attribute = arg

    def __repr__(self) -> str:
        return (
            f"<class {self.__class__.__name__}("
            f"{self.attribute!r}) "
            f"at {hex(id(self))}>"
        )

Overwriting singleton.py


## Unit Test

Using `pytest`

In [1]:
%%writefile ../../tests/unit/test_singleton.py
from patterns import Singleton, FooTone


def test_singleton(tmpdir):
    assert "<class 'patterns.singleton.singleton.FooTone'>" not in Singleton._instances
    assert len(Singleton._instances) == 0

    foo1 = FooTone(tmpdir.join("some_attr"))
    foo2 = FooTone(tmpdir.join("diff_attr"))

    assert foo1 == foo2

    foo2.attribute = tmpdir.join("another_attr")
    assert foo1.attribute == foo2.attribute

    assert len(Singleton._instances) == 1
    assert "<class 'patterns.singleton.singleton.FooTone'>" in Singleton._instances

Overwriting ../../tests/unit/test_singleton.py


In [1]:
%run -m pytest ../../tests/unit/test_singleton.py

.                                                                        [100%]
1 passed in 0.02s


### old test using `unittest`

In [48]:
%%writefile test_singleton-ipynb.py
import unittest
from singleton import Singleton, FooTone


class TestSingleton(unittest.TestCase):
    def test_singleton(self):
        # print(Singleton._instances)
        # self.assertEqual(len(Singleton._instances), 0)
        # self.assertNotIn("<class 'patterns.singleton.singleton.FooTone'>", Singleton._instances)

        foo1 = FooTone("some_attr")
        foo2 = FooTone("diff_attr")

        self.assertEqual(foo1, foo2)

        foo2.attribute = "another_attr"
        self.assertEqual(foo1.attribute, foo2.attribute)

        self.assertEqual(len(Singleton._instances), 1)
        self.assertIn("<class 'singleton.FooTone'>", Singleton._instances)
        print('Success: test_singleton')


if __name__ == '__main__':
    unittest.main()

Overwriting test_singleton-ipynb.py


In [49]:
%run -i test_singleton-ipynb.py

.

{"<class 'singleton.FooTone'>": <class FooTone('another_attr') at 0x7f3ca94d9400>}
Success: test_singleton



----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


# Project implementation

A simple project implementation using the Singleton pattern.

## `logger.py`

A simple logging module that keeps track of the `.log` file state, writes specific logging messages to the log file accordingly, and reads the contents of the log file.

In [1]:
%%writefile logger.py
from patterns import Singleton


class Logger(metaclass=Singleton):
    def __init__(self, file_name: str) -> None:
        self.file_name = file_name

    def _write_log(self, level: str, msg: str) -> None:
        with open(self.file_name, "a") as log_file:
            log_file.write(f"[{level}] {msg}\n")

    def critical(self, msg: str) -> None:
        self._write_log("CRITICAL", msg)
    
    def error(self, msg: str) -> None:
        self._write_log("ERROR", msg)

    def warn(self, msg: str) -> None:
        self._write_log("WARNING", msg)

    def info(self, msg: str) -> None:
        self._write_log("INFO", msg)

    def debug(self, msg: str) -> None:
        self._write_log("DEBUG", msg)

    def read(self) -> str:
        with open(self.file_name, "r") as log_file:
            return log_file.read()

    def __repr__(self) -> str:
        return (
            f"<class {self.__class__.__name__}("
            f"{self.file_name!r}) "
            f"at {hex(id(self))}>"
        )



if __name__ == '__main__':
    # print("<class '__main__.Logger'>" in Singleton._instances)
    # print(f"{len(Singleton._instances)}\n")

    # logger1 = Logger("class_logger.log")
    # logger2 = Logger("tst.txt")
    # print(logger1 is logger2)
    
    # logger2.file_name = "test/another_logger.log"
    # print(logger1.file_name is logger2.file_name)

    # print(f"\n{len(Singleton._instances)}")
    # print("<class '__main__.Logger'>" in Singleton._instances)
    print(Logger("class_logger.log"))

Overwriting logger.py


## Functional Test

Using `pytest`

In [5]:
%%writefile ../../tests/func/test_logger.py
from patterns import Logger


def test_logger(tmpdir_factory):
    tmp = tmpdir_factory.mktemp("func")
    logger = Logger(tmp.join("class_logger.log"))

    logger.critical("This is a critical message")
    logger.error("This is an error message")
    logger.warn("This is a warning message")
    logger.info("This is an info message")
    logger.debug("This is a debugging message")

    assert logger.read() == (
                                f"[CRITICAL] This is a critical message\n"
                                f"[ERROR] This is an error message\n"
                                f"[WARNING] This is a warning message\n"
                                f"[INFO] This is an info message\n"
                                f"[DEBUG] This is a debugging message\n"
                            )

Overwriting ../../tests/func/test_logger.py


In [1]:
%run -m pytest ../../tests/func/test_logger.py

.                                                                        [100%]
1 passed in 0.02s
